In [ ]:
import numpy as np
import msmrd2
from msmrd2.markovModels import continuousTimeMarkovStateModel as ctmsm
from msmrd2.integrators import overdampedLangevinMarkovSwitch as odLangevinMS 

In [ ]:
# Define continuous-time MSM
MSMtype = 1
ratematrix = np.array([[-5.0,2.0,3.0],[1.0,-2.0,1.0],[3.3,5.4,-8.7]])
seed = 0 # Seed = -1 used random device as seed
markovModel = ctmsm(MSMtype, ratematrix, seed)
Dlist = np.array([1.0, 3.0, 2.0])
Drotlist = np.array([0.5, 0.2, 0.1])
markovModel.setD(Dlist)
markovModel.setDrot(Drotlist)

In [ ]:
# Particle definition
ptype = 0
pstate = 0
D = Dlist[pstate]
Drot = Drotlist[pstate]
position = np.array([1,1,1])
orientation = np.array([1,0,0,0])
part1 = msmrd2.particle(ptype, pstate, D, Drot, position, orientation)

In [ ]:
# Over-damped Langevin integrator with Markovian switch definition
dt = 0.05 # should be smaller than Gillespie inverse transition rates
seed = 0 # Seed = -1 used random device as seed
bodytype = 'rigidbody'
integrator1 = odLangevinMS(markovModel, dt, seed, bodytype) 
# Create second integrator for list integration
integrator2 = odLangevinMS(markovModel, dt, seed, bodytype) 

In [ ]:
# Integrate only one particle and print position and orientation
timesteps = 50
# Need to define a one particle list (note part1 object is not modified when list is modified)
part1list = msmrd2.integrators.particleList([part1])
print('{:<10s}{:<15s}{:<15s}{:<15s}{:<15s}{:<40s}'.format("Iteration", "Time", "State", "D", "Drot", "Position 1"))
for i in range(timesteps):
    print('{:<10d}{:<15f}{:<15d}{:<15f}{:<15f}{:<40s}'
          .format(i, integrator1.clock, part1list[0].state, part1list[0].D, part1list[0].Drot, str(part1list[0].position)))
    integrator1.integrate(part1list)

In [ ]:
# For a list of particles
pairInteractions = False
position2 = np.array([2,2,2])
part1 = msmrd2.particle(ptype, pstate, D, Drot, position, orientation)
part2 = msmrd2.particle(ptype, pstate, D, Drot, position2, orientation)
partlist = msmrd2.integrators.particleList([part1,part2])
# Integrate particle list and print only positions (with integratelist)
t = 0
timesteps = 50
print('{:<10s}{:^10s}{:<40s}{:^15s}{:<30s}'.format("Time", "State 1", "Position 1", "State 2", "Position2"))
for i in range(timesteps):
    print('{:<10f}{:^10d}{:<40s}{:^15d}{:<30s}'
          .format(integrator2.clock, partlist[0].state, str(partlist[0].position), partlist[1].state, str(partlist[1].position)))
    integrator2.integrate(partlist)